In [17]:
!ls /share/apps/openmpi/4.1.1/intel/bin

aggregate_profile.pl  mpif90	    orte-clean	 oshc++		 shmemCC
mpiCC		      mpifort	    orte-info	 oshcc		 shmemc++
mpic++		      mpirun	    orte-server  oshcxx		 shmemcc
mpicc		      ompi-clean    ortecc	 oshfort	 shmemcxx
mpicxx		      ompi-server   orted	 oshmem_info	 shmemfort
mpiexec		      ompi_info     orterun	 oshrun		 shmemrun
mpif77		      opal_wrapper  oshCC	 profile2mat.pl


In [1]:
!module load cuda/11.6.2
!module load cudnn/8.6.0.163-cuda11
!module load openmpi/intel/4.1.1

In [2]:
!pip install -q --use-feature=2020-resolver pynvml zstandard datasets psutil transformers torch bitsandbytes accelerate loralib deepspeed peft
!echo "Hostname: $(hostname)"
!echo "Processor: $(lscpu | grep 'Model name' | awk -F ':' '{print $2}' | xargs)"
!echo "RAM: $(free -h | grep 'Mem:' | awk '{print $4}')"

You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Hostname: gr052.hpc.nyu.edu
Processor: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz
RAM: 146Gi


In [20]:
!export CC=/share/apps/openmpi/4.1.1/intel/bin/mpicc
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/share/apps/openmpi/4.1.1/intel/bin/mpicc:/share/apps/openmpi/4.1.1/intel/lib:/share/apps/mvapich2/2.3.6/intel/lib
!pip install mpi4py

Defaulting to user installation because normal site-packages is not writeable
  Using cached mpi4py-3.1.4.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Command errored out with exit status 1:
   command: /share/apps/python/3.8.6/intel/bin/python /share/apps/python/3.8.6/intel/lib/python3.8/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /state/partition1/job-32915051/tmpip3s6pdi
       cwd: /state/partition1/job-32915051/pip-install-vmo4xya4/mpi4py
  Complete output (148 lines):
  running bdist_wheel
  running build
  running build_src
  running build_py
  creating build
  creating build/lib.linux-x86_64-cpython-38
  creating build/lib.linux-x86_64-cpython-38/mpi4py
  copying src/mpi4py/__init__.py -> build/lib.linux-x86_64-cpython-38/mpi4py
  copying src/mpi4py/__main__.py -> build/lib.linux-x86_64-cpython-38/mpi4py
  copying src/mpi4py/bench.py -> build/lib.linux-x86_6

You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [1]:
!echo "GPU: $(nvidia-smi --query-gpu=name --format=csv,noheader)"
!echo "GPU Memory: $(nvidia-smi | grep MiB |  awk '{print $9 $10 $11}')"

GPU: Quadro RTX 8000
GPU Memory: 0MiB/46080MiB


In [3]:
# Change net ID here to use your scratch folder
ENV = "dev"
NET_ID = "vgn2004"
ROOT_PATH = f"/scratch/{NET_ID}/fine_tuning"

# Global configurations
config = {
    "DATASET_URL": "https://the-eye.eu/public/AI/pile_v2/data",
    "DATASET_NAME": "NIH_ExPORTER_awarded_grant_text",
    "NUM_WORKERS": 8,
    "DATASET_SPLIT_RATIO": 0.9,
    "PADDING_STRATEGY": "max_length",
    "MAX_TOKENS": 128,
    "MODEL_NAME": "gpt2",
    "TOKENIZED_NAME": "autos",
    "BATCH_SIZE": 256,
    "NUM_EPOCHS": 25,
    "LEARNING_RATE": 0.001,
    "EPSILON": 1e-8,
    "SAMPLING_INTERVAL": 64,
    "CHECKPOINTING_INTERVAL": 100,
    "VALIDATION_INTERVAL": 500,
    "GRADIENT_ACCUMULATION_STEPS": 8,
}

deepspeed_config = {
    "fp16": {"enabled": True, "auto_cast": True},
    "optimizer": {
        "type": "AdamW",
        "params": {"lr": 3e-5, "betas": [0.9, 0.999], "eps": 1e-8},
    },
    "activation_checkpointing": {
        "partition_activations": True,
        "cpu_checkpointing": True,
        "contiguous_memory_optimization": True,
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {"warmup_min_lr": 0, "warmup_max_lr": 3e-5, "warmup_num_steps": 50},
    },
    "zero_optimization": {
        "stage": 3,
        "overlap_comm": True,
        "contiguous_gradients": True,
    },
    "gradient_accumulation_steps": 8,
    "steps_per_print": 100,
    "train_micro_batch_size_per_gpu": 16,
}

# ds_config = {
#     "fp16": {"enabled": True},
#     "zero_optimization": {
#         "stage": 3,
#         "allgather_partitions": True,
#         "allgather_bucket_size": 5e8,
#         "overlap_comm": True,
#         "reduce_scatter": True,
#         "reduce_bucket_size": 5e8,
#         "contiguous_gradients": True
#     },
#     "optimizer": {
#         "type": "AdamW",
#         "params": {
#             "lr": 3e-5,
#             "betas": [0.9, 0.999],
#             "eps": 1e-8
#         }
#     },
#     "scheduler": {
#         "type": "WarmupLR",
#         "params": {
#             "warmup_min_lr": 0,
#             "warmup_max_lr": 3e-5,
#             "warmup_num_steps": 1000
#         }
#     },
#     "steps_per_print": 100,
#     "gradient_accumulation_steps": 1,
#     "train_batch_size": 16,
#     "train_micro_batch_size_per_gpu": 4,
#     "wall_clock_breakdown": False
# }

# Ensure that packages can be found
import sys

sys.path.insert(0, f"/home/{NET_ID}/.local/lib/python3.8/site-packages")

# Ensure that GPU can be found
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

# Setup logging
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s"
)

# Packages for profiling
import random
import psutil
from time import time
from tqdm import tqdm
import tqdm.notebook as tq
from pynvml import *

# Packages for data loading
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import DataCollatorForLanguageModeling

# Core packages
import torch

torch.cuda.empty_cache()
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_math_sdp(False)
logging.info(f"Is Flash Attention Enabled: {torch.backends.cuda.flash_sdp_enabled()}")
logging.info(
    f"Is Mem Efficient SDP Enabled: {torch.backends.cuda.mem_efficient_sdp_enabled()}"
)
logging.info(f"Is Math SDP Enabled: {torch.backends.cuda.math_sdp_enabled()}")

import torch.distributed as dist

# dist.init_process_group(backend=None)

import deepspeed
import bitsandbytes.optim as bnb_optim

# from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers.optimization import Adafactor
from transformers.deepspeed import HfDeepSpeedConfig


# Unused imports (TODO)
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig


# Get GPU Utilization
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    logging.info(f"GPU memory occupied: {info.used//1024**2} MB.")


# Returns RAM usage in MB
def get_ram_usage():
    return psutil.Process().memory_info().rss / (1024 * 1024)


# Takes a batch of inputs and runs the tokenizer on them
def tokenize_function(examples, tokenizer):
    return tokenizer(
        examples["text"],
        padding=config["PADDING_STRATEGY"],
        truncation=True,
        max_length=config["MAX_TOKENS"],
        return_attention_mask=True,
    )


# Tokenizes dataset and creates train and validation split
def preprocess_data(dataset, tokenizer):
    tokenized_dataset_path = f"{ROOT_PATH}/datasets/tokenized_{config['DATASET_NAME']}_{config['TOKENIZED_NAME']}"
    train_dataset_path = f"{tokenized_dataset_path}_train"
    valid_dataset_path = f"{tokenized_dataset_path}_valid"
    if os.path.exists(train_dataset_path) and os.path.exists(valid_dataset_path):
        train_dataset = load_from_disk(train_dataset_path)
        valid_dataset = load_from_disk(valid_dataset_path)
        return train_dataset, valid_dataset

    logger.info(f"Tokenizing the dataset...")
    start_time = time()
    try:
        tokenized_dataset = load_from_disk(tokenized_dataset_path)
    except Exception as e:
        logging.error(e)
        tokenized_dataset = dataset.map(
            tokenize_function,
            fn_kwargs={"tokenizer": tokenizer},
            batched=True,
            num_proc=8,
            remove_columns=["text", "meta"],
        )
        tokenized_dataset.save_to_disk(tokenized_dataset_path)

    elapsed_time = time() - start_time
    logger.info(f"Time taken to tokenize the dataset: {elapsed_time:.2f} seconds")

    logger.info(f"Splitting the dataset...")
    start_time = time()

    if os.path.exists(train_dataset_path) and os.path.exists(valid_dataset_path):
        train_dataset = load_from_disk(train_dataset_path)
        valid_dataset = load_from_disk(valid_dataset_path)
    else:
        train_size = int(config["DATASET_SPLIT_RATIO"] * len(tokenized_dataset))
        datasets = DatasetDict(
            {
                "train": Dataset.from_dict(tokenized_dataset[:train_size]),
                "valid": Dataset.from_dict(tokenized_dataset[train_size:]),
            }
        )
        train_dataset = datasets["train"]
        valid_dataset = datasets["valid"]
        train_dataset.save_to_disk(train_dataset_path)
        valid_dataset.save_to_disk(valid_dataset_path)
    elapsed_time = time() - start_time
    logger.info(
        f"Time taken to split the datasets (or load pre-split datasets): {elapsed_time:.2f} seconds"
    )

    return train_dataset, valid_dataset


# Creates data loaders
def create_dataloaders(train_dataset, valid_dataset, data_collator):
    logger.info(f"Creating data loaders...")
    start_time = time()
    train_dataloader = DataLoader(
        train_dataset,
        sampler=RandomSampler(train_dataset),
        batch_size=config["BATCH_SIZE"],
        num_workers=config["NUM_WORKERS"],
        collate_fn=data_collator,
    )
    valid_dataloader = DataLoader(
        valid_dataset,
        sampler=SequentialSampler(valid_dataset),
        batch_size=config["BATCH_SIZE"],
        num_workers=config["NUM_WORKERS"],
        collate_fn=data_collator,
    )
    elapsed_time = time() - start_time
    logging.info(f"Time taken to create data loaders: {elapsed_time:.2f} seconds")
    return train_dataloader, valid_dataloader


# Fetches tokenizer relevant to the model
def create_or_load_tokenizer(checkpointed_path=None):
    if checkpointed_path:
        tokenizer = AutoTokenizer.from_pretrained(checkpointed_path)
    else:
        tokenizer = AutoTokenizer.from_pretrained(
            config["MODEL_NAME"], cache_dir=f"{ROOT_PATH}/datasets"
        )
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
    return tokenizer


# Data preparation
def run_data_pipeline(tokenizer, load_from_file=False):
    # Measure how much RAM is being used before anything runs
    ram_usage = get_ram_usage()
    logging.info(f"Baseline: RAM used: {ram_usage:.2f} MB")

    # Load data, either from url or from datasets folder
    data_file_url = f"{config['DATASET_URL']}/{config['DATASET_NAME']}.jsonl.zst"
    try:
        if load_from_file:
            raise Exception
        dataset = load_dataset(
            "json",
            data_files=data_file_url,
            num_proc=config["NUM_WORKERS"],
            split="train",
            cache_dir=f"{ROOT_PATH}/datasets",
        )
    except Exception as e:
        logging.error(e)
        dataset = load_dataset(
            "json",
            data_files=f"{ROOT_PATH}/datasets/{config['DATASET_NAME']}.jsonl.zst",
            num_proc=config["NUM_WORKERS"],
            split="train",
            cache_dir=f"{ROOT_PATH}/datasets",
        )

    # Measurements relevant to the dataset
    ram_usage = get_ram_usage()
    logging.info(f"RAM used: {ram_usage:.2f} MB")
    logging.info(f"Dataset sample: {dataset[10]}")
    size_gb = dataset.dataset_size / (1024**3)
    logging.info(f"Dataset size (cache file) : {size_gb:.2f} GB")

    # Fetch a tokenizer and tokenize + split the dataset
    train_dataset, valid_dataset = preprocess_data(dataset, tokenizer)

    # Create a data collator and use it to make data loaders
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    train_dataloader, valid_dataloader = create_dataloaders(
        train_dataset, valid_dataset, data_collator
    )

    return {
        "TRAIN_DATASET": train_dataset,
        "VALIDATION_DATASET": valid_dataset,
        "TRAIN_DATALOADER": train_dataloader,
        "VALIDATION_DATALOADER": valid_dataloader,
        "TOKENIZER": tokenizer,
    }


# Create model
def create_or_load_model(checkpointed_path=None, quantized=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if checkpointed_path:
        model = AutoModelForCausalLM.from_pretrained(checkpointed_path)
        model.to(device)
    else:
        configuration = AutoConfig.from_pretrained(config["MODEL_NAME"])
        if quantized:
            model = AutoModelForCausalLM.from_pretrained(
                config["MODEL_NAME"],
                config=configuration,
                device_map="auto",
                load_in_8bit=True,
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                config["MODEL_NAME"], config=configuration
            )
            model.to(device)

    # Measurements relevant to the model
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info(f"Model: {config['MODEL_NAME']}")
    logger.info(f"Total Parameters: {total_params / 1e6:.2f} million")
    logger.info(f"Trainable Parameters: {trainable_params / 1e6:.2f} million")
    logger.info(f"Memory Footprint: {model.get_memory_footprint() / 1e6:,} MB")
    logger.info(f"Model is on device: {model.device}")
    return model, device


# Use the model to generate text
def inference(model, tokenizer, device, max_tokens=128):
    # Put the model in eval mode
    model.eval()

    # Generate a sequence of text tokens
    inputs = tokenizer.encode(tokenizer.eos_token + "This", return_tensors="pt").to(
        device
    )
    output_sequence = model.generate(
        inputs,
        bos_token_id=tokenizer.bos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        max_length=max_tokens,
        top_p=0.95,
        num_return_sequences=1,
        use_cache=False,
    )

    # Decode the tokens to text
    generated_text = (
        tokenizer.decode(output_sequence[0], clean_up_tokenization_spaces=True)
        .replace("\n", "")
        .replace("\t", " ")
    )

    # Put the model back into train mode
    model.train()
    return generated_text


# Evaluate the model on a data loader
def validate(model, device, valid_dataloader):
    #     counter = 0
    model.eval()
    total_eval_loss = 0.0
    for batch in tqdm(valid_dataloader):
        #         counter+=1
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss
        total_eval_loss += loss.item()
    #         if counter==10:
    #             break

    avg_eval_loss = total_eval_loss / len(valid_dataloader)
    perplexity = torch.exp(torch.tensor(avg_eval_loss)).item()
    model.train()
    return avg_eval_loss, perplexity


# Train the model
def train(model, device, data_dict):
    model_save_path = f"{ROOT_PATH}/models/{ENV}/fine_tuned_{config['MODEL_NAME']}_{config['DATASET_NAME']}_{config['TOKENIZED_NAME']}"

    # Setup logging
    log_save_path = f"{ROOT_PATH}/logs/{ENV}/{config['MODEL_NAME']}_{config['DATASET_NAME']}_{config['TOKENIZED_NAME']}"
    if not os.path.exists(log_save_path):
        os.makedirs(log_save_path)
    with open(f"{log_save_path}/training.log", "w+") as f:
        f.write("epoch\tbatch\ttrain\tloss\tgenerated_text\n")
    with open(f"{log_save_path}/validation.log", "w+") as f:
        f.write("epoch\tbatch\tvalidation_loss\tperplexity\n")

    train_dataloader = data_dict["TRAIN_DATALOADER"]
    valid_dataloader = data_dict["VALIDATION_DATALOADER"]
    tokenizer = data_dict["TOKENIZER"]

    engine, optimizer, _, _ = deepspeed.initialize(
        config=deepspeed_config,
        model=model,
        model_parameters=model.parameters(),
        dist_init_required=False,
    )

    #     optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, warmup_init=False, lr=config["LEARNING_RATE"])
    #     optimizer = torch.optim.Adam(model.parameters(), lr=config["LEARNING_RATE"],eps=config["EPSILON"])
    #     lr_scheduler = get_linear_schedule_with_warmup(optimizer,
    #                                                    num_warmup_steps=100,
    #                                                    num_training_steps=config["NUM_EPOCHS"] * len(train_dataloader))

    model.train()
    #     scaler = torch.cuda.amp.GradScaler()

    # Go through each epoch
    for epoch in tqdm(range(1, config["NUM_EPOCHS"] + 1)):
        counter = 0
        running_loss = 0.0
        logging.info(f"Epoch: {epoch}/{config['NUM_EPOCHS']}")

        # Go through each batch in the data loader
        for index, batch in tqdm(
            enumerate(train_dataloader), total=len(train_dataloader)
        ):
            optimizer.zero_grad()
            if counter < 5:
                print_gpu_utilization()
                counter += 1
            #             !echo "GPU Memory: $(nvidia-smi | grep MiB |  awk '{print $9 $10 $11}')"
            # Measure average loss and sample an output from the model, at each sampling interval
            if (index + 1) % config["SAMPLING_INTERVAL"] == 0:
                avg_loss = running_loss / config["SAMPLING_INTERVAL"]
                logging.info(
                    f"Batch {index+1}/{len(train_dataloader)}, Loss: {avg_loss:.4f}"
                )

                #                 generated_text = inference(model, tokenizer, device)
                #                 logging.info(f"Text:\n{generated_text}")

                with open(f"{log_save_path}/training.log", "a") as f:
                    f.write(f"{epoch}\t{index+1}\t{avg_loss}\t{generated_text}\n")

                running_loss = 0.0

            # Save the model at each checkpointing interval
            if (index + 1) % config["CHECKPOINTING_INTERVAL"] == 0:
                logging.info(
                    f"Checkpointing model at epoch={epoch} and batch={index+1}\n"
                )

                checkpointing_path = f"{model_save_path}_{epoch}_{index}"
                model.save_pretrained(checkpointing_path)
                tokenizer.save_pretrained(checkpointing_path)

            # Validate the model at each validation interval
            if (index + 1) % config["VALIDATION_INTERVAL"] == 0:
                logging.info("Running Validation...")
                avg_eval_loss, perplexity = validate(model, device, valid_dataloader)
                logging.info(
                    f"Batch {index + 1}/{len(train_dataloader)}, Validation Loss: {avg_eval_loss:.4f}, Perplexity: {perplexity:.2f}"
                )
                with open(f"{log_save_path}/validation.log", "a") as f:
                    f.write(f"{epoch}\t{index+1}\t{avg_eval_loss}\t{perplexity}\n")

            # Backpropagation
            batch = {k: v.to(device) for k, v in batch.items()}
            loss = engine(**batch)
            engine.backward(loss)
            engine.step()
    #             with torch.cuda.amp.autocast(dtype=torch.float16):
    #                 outputs = model(**batch)
    #                 loss = outputs.loss
    #                 loss = loss / config["GRADIENT_ACCUMULATION_STEPS"]

    # Using a scaler
    #             scaler.scale(loss).backward()
    #             loss.backward()

    #             running_loss += loss.item()*config["GRADIENT_ACCUMULATION_STEPS"]

    #             if (index+1) % config["GRADIENT_ACCUMULATION_STEPS"] == 0:
    #                 scaler.step(optimizer)
    #                 scaler.update()
    #                 lr_scheduler.step()
    #                 optimizer.step()

    # After all epochs are completed, save the final model and tokenier
    #     model.save_pretrained(model_save_path)
    engine.save_checkpoint(model_save_path, model_save_path)
    tokenizer.save_pretrained(model_save_path)


if __name__ == "__main__":
    os.environ["RANK"] = "0"
    os.environ["WORLD_SIZE"] = "1"
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    os.environ["LOCAL_RANK"] = "0"
    torch.cuda.set_device(0)  # Set the GPU device you'd like to use
    deepspeed.init_distributed(
        dist_backend="nccl", rank=0, world_size=1
    )  # Initialize the distributed backend

    print_gpu_utilization()
    #     checkpointed_path = f"/scratch/{NET_ID}/finetunedgpt2_5epochs"
    checkpointed_path = None
    dschf = HfDeepSpeedConfig(deepspeed_config)
    tokenizer = create_or_load_tokenizer(checkpointed_path=checkpointed_path)
    data_dict = run_data_pipeline(tokenizer, load_from_file=False)
    model, device = create_or_load_model(checkpointed_path=checkpointed_path)
    model.gradient_checkpointing_enable()

    #     generated_text = inference(model, tokenizer, device)
    #     logging.info(f"Initial Text:\n{generated_text}")

    train(model, device, data_dict)

2023-04-30 01:35:04,325 - INFO - <module>:129 - Is Flash Attention Enabled: True
2023-04-30 01:35:04,326 - INFO - <module>:130 - Is Mem Efficient SDP Enabled: False
2023-04-30 01:35:04,326 - INFO - <module>:131 - Is Math SDP Enabled: False
2023-04-30 01:35:04,329 - INFO - print_gpu_utilization:154 - GPU memory occupied: 4648 MB.
2023-04-30 01:35:04,492 - INFO - run_data_pipeline:252 - Baseline: RAM used: 2291.26 MB
2023-04-30 01:35:04,753 - WARNING - download_and_prepare:817 - Found cached dataset json (/scratch/vgn2004/fine_tuning/datasets/json/default-3ed31d147447149f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
2023-04-30 01:35:04,767 - INFO - run_data_pipeline:274 - RAM used: 2296.86 MB
2023-04-30 01:35:04,768 - INFO - run_data_pipeline:275 - Dataset sample: {'meta': {'APPLICATION_ID': 100075}, 'text': "ACF's Office of Refugee Resettlement (ORR) administers a variety of social service programs intended to connect newly resettled refugees with critical res

[2023-04-30 01:35:05,335] [INFO] [partition_parameters.py:454:__exit__] finished initializing model with 0.49B parameters


2023-04-30 01:35:05,516 - INFO - create_or_load_model:311 - Model: gpt2
2023-04-30 01:35:05,517 - INFO - create_or_load_model:312 - Total Parameters: 0.00 million
2023-04-30 01:35:05,517 - INFO - create_or_load_model:313 - Trainable Parameters: 0.00 million
2023-04-30 01:35:05,519 - INFO - create_or_load_model:314 - Memory Footprint: 12.58296 MB
2023-04-30 01:35:05,519 - INFO - create_or_load_model:315 - Model is on device: cuda:0


[2023-04-30 01:35:05,520] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.9.1, git-hash=unknown, git-branch=unknown
[2023-04-30 01:35:05,529] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Installed CUDA version 11.8 does not match the version torch was compiled with 11.7 but since the APIs are compatible, accepting this combination
Time to load fused_adam op: 0.0013434886932373047 seconds
[2023-04-30 01:35:05,655] [INFO] [logging.py:96:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adamw as basic optimizer
[2023-04-30 01:35:05,663] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2023-04-30 01:35:05,664] [INFO] [utils.py:51:is_zero_supported_optimizer] Checking ZeRO support for optimizer=FusedAdam type=<class 'deepspeed.ops.adam.fused_adam.FusedAdam'>
[2023-04-30 01:35:05,665] [INFO] [logging.py:96:log_dist] [Rank 0] Creating torch.float16 ZeRO stage 3 optimizer


Using /home/vgn2004/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module fused_adam, skipping build step...
Loading extension module fused_adam...


...

[2023-04-30 01:35:07,025] [INFO] [config.py:957:print]   eigenvalue_enabled ........... False
[2023-04-30 01:35:07,026] [INFO] [config.py:957:print]   eigenvalue_gas_boundary_resolution  1
[2023-04-30 01:35:07,026] [INFO] [config.py:957:print]   eigenvalue_layer_name ........ bert.encoder.layer
[2023-04-30 01:35:07,026] [INFO] [config.py:957:print]   eigenvalue_layer_num ......... 0
[2023-04-30 01:35:07,027] [INFO] [config.py:957:print]   eigenvalue_max_iter .......... 100
[2023-04-30 01:35:07,027] [INFO] [config.py:957:print]   eigenvalue_stability ......... 1e-06
[2023-04-30 01:35:07,027] [INFO] [config.py:957:print]   eigenvalue_tol ............... 0.01
[2023-04-30 01:35:07,027] [INFO] [config.py:957:print]   eigenvalue_verbose ........... False
[2023-04-30 01:35:07,028] [INFO] [config.py:957:print]   elasticity_enabled ........... False
[2023-04-30 01:35:07,028] [INFO] [config.py:957:print]   flops_profiler_config ........ {
    "enabled": false, 
    "profile_step": 1, 
    "modul

Using /home/vgn2004/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...


Time to load utils op: 0.006386995315551758 seconds


  0%|          | 0/25 [00:00<?, ?it/s]2023-04-30 01:35:07,048 - INFO - train:399 - Epoch: 1/25

  0%|          | 0/3466 [00:00<?, ?it/s]2023-04-30 01:35:16,257 - INFO - print_gpu_utilization:154 - GPU memory occupied: 8134 MB.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
  0%|          | 0/25 [00:10<?, ?it/s]


AttributeError: 'CausalLMOutputWithCrossAttentions' object has no attribute 'float'